In [1]:
import prop
import pandas as pd
import matplotlib.pyplot as plt
from modules import pump, regen, heater, turbine, condenser

#Таблица с потоками и блоками:
streams = pd.read_excel("streams.xlsx",index_col=0)
blocks  = pd.read_excel("blocks.xlsx" ,index_col=0)

#Параметры нагревающей среды:
gas  = 'REFPROP::N2[0.78]&O2[0.1]&CO2[0.02]&H2O[0.1]' #Состав газа
Tin  = 183.6                                           #Температура газов на входе
Pgas = 0.1                                             #Давление газов
Ggas = 509                                             #Расход газов
Tout = 80                                              #Температура газов на выходе

#Параметры охлаждающей среды:
fluidcond  = "REFPROP::WATER" #Охлаждающая среда
Tfluidcond = 15               #Температура охл. среды
Pfluidcond = 0.5              #Давление охл. среды

#Параметры ОЦР:
fluid   = "REFPROP::R236ea"       #Рабочая среда цикла
KPDpump = 0.85                 #КПД насоса
KPDturb = 0.85                 #КПД турбины
DTheat  = 10                   #Минимальный напор в нагревателе
DTcond  = 5                    #Минимальный напор в конденсаторе
Pcond   = prop.t_q(30,0,fluid)['P']               #Давление в конденсаторе

#Первое приближение

Ppump   = 4.6

dTreg   = 22.5
for j in range(20):

    dPreg1  = 0.0
    for i in range(16):

        dPreg2  = dPreg1


        streams = pd.read_excel("streams.xlsx",index_col=0)
        blocks  = pd.read_excel("blocks.xlsx" ,index_col=0)

        #Внесение входных данных в таблицу:
        streams.loc["IN-HEAT", "T":"Q"] = [Tin,Pgas,prop.t_p(Tin,Pgas,gas)["H"],prop.t_p(Tin,Pgas,gas)["S"],Ggas,1]
        streams.loc["IN-COND", "T":"Q"] = [Tfluidcond,Pfluidcond,prop.t_p(Tfluidcond,Pfluidcond,fluidcond)["H"],prop.t_p(Tfluidcond,Pfluidcond,fluidcond)["S"],1000,0]

        #Начало расчёта - задание расхода на первой итерации:
        G = Ggas*(prop.t_p(Tin,Pgas,gas)["H"]-prop.t_p(Tout,Pgas,gas)["H"])/(prop.t_p(Tin,Ppump,fluid)["H"]-prop.t_p(Tout,Ppump,fluid)["H"])

        #Поток перед насосом:
        streams.loc["COND-PUMP", "T":"Q"] = [prop.p_q(Pcond,0,fluid)["T"],Pcond,prop.p_q(Pcond,0,fluid)['H'],prop.p_q(Pcond,0,fluid)['S'],G,0]

        #Итеративный расчет для сведения баланса:
        for i in range(1000):
            #Расчет цикла:
            pump.calc     ("COND-PUMP", "PUMP-REG",                          Ppump,  KPDpump,     fluid, streams, blocks)
            regen.calc    ("TURB-REG",  "REG-COND",  "PUMP-REG","REG-HEAT",  dTreg, dPreg1,dPreg2,fluid, streams, blocks)
            heater.calc   ("IN-HEAT",   "HEAT-OUT",  "REG-HEAT","HEAT-TURB", Tout,   DTheat, gas, fluid, streams, blocks)
            turbine.calc  ("HEAT-TURB", "TURB-REG",                          Pcond+dPreg1,  KPDturb,fluid, streams, blocks)
            regen.calc    ("TURB-REG",  "REG-COND",  "PUMP-REG","REG-HEAT",  dTreg, dPreg1,dPreg2,fluid, streams, blocks)
            condenser.calc("REG-COND",  "COND-PUMP", "IN-COND" ,"COND-OUT",  DTcond, fluidcond,   fluid, streams, blocks)
            Qbalance = blocks.loc['HEATER','Q']+blocks.loc['PUMP','N']-blocks.loc['CONDENSER','Q']-blocks.loc['TURBINE','N']
    #         print(Qbalance)
            if abs(Qbalance) < 1:
                break
        fi = (streams.loc["TURB-REG", "T"] -streams.loc["REG-COND", "T"]) / (streams.loc["TURB-REG", "T"]-streams.loc["PUMP-REG", "T"])
        
        KPD = (blocks.loc['TURBINE','N']-blocks.loc['PUMP','N'])/blocks.loc['HEATER','Q']


        print(round(dTreg,5),round(fi,5),round(dPreg1,2), round(KPD*100,5))
        dPreg1 = dPreg1 + 0.01
    dTreg = dTreg+1

22.5 0.3667 0.0 16.78467
22.5 0.37871 0.01 16.53584
22.5 0.38993 0.02 16.29554
22.5 0.40049 0.03 16.06244
22.5 0.41044 0.04 15.83642
22.5 0.41984 0.05 15.61703
22.5 0.42873 0.06 15.40387
22.5 0.43718 0.07 15.19656
22.5 0.44521 0.08 14.99478
22.5 0.45285 0.09 14.79821
22.5 0.46016 0.1 14.60658
22.5 0.46714 0.11 14.41961
22.5 0.47382 0.12 14.23708
22.5 0.48023 0.13 14.05877
22.5 0.48639 0.14 13.88446
22.5 0.49231 0.15 13.71398
23.5 0.3432 0.0 16.73463
23.5 0.35556 0.01 16.48658
23.5 0.36714 0.02 16.24665
23.5 0.378 0.03 16.01467
23.5 0.38825 0.04 15.78937
23.5 0.39794 0.05 15.57069
23.5 0.40711 0.06 15.35821
23.5 0.41582 0.07 15.15157
23.5 0.4241 0.08 14.95043
23.5 0.43199 0.09 14.75449
23.5 0.43952 0.1 14.56348
23.5 0.44672 0.11 14.37712
23.5 0.45362 0.12 14.19519
23.5 0.46024 0.13 14.01745
23.5 0.46659 0.14 13.84372
23.5 0.47271 0.15 13.67379
24.5 0.32003 0.0 16.68496
24.5 0.33274 0.01 16.43769
24.5 0.34464 0.02 16.19852
24.5 0.35583 0.03 15.96688
24.5 0.36636 0.04 15.74267
24.5 0.3763

41.5 0.01992 0.03 15.21207
41.5 0.03433 0.04 14.99862
41.5 0.04797 0.05 14.79169
41.5 0.0609 0.06 14.59107
41.5 0.07323 0.07 14.3957
41.5 0.08499 0.08 14.20556
41.5 0.09623 0.09 14.02036
41.5 0.10698 0.1 13.83982
41.5 0.11729 0.11 13.6637
41.5 0.12722 0.12 13.49143
41.5 0.13674 0.13 13.32345
41.5 0.1459 0.14 13.15926
41.5 0.15473 0.15 12.99869
